In [1]:
from Bio.Seq import reverse_complement
from collections import Counter
import itertools
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import auc
from sklearn.model_selection import cross_validate

from tqdm import tqdm_notebook as tqdm

In [2]:
def load_fasta(path):
    with open(path) as f:
        f.readline()
        content = f.read()
        
    for ch in [' ', '\n'] :
        content = content.replace(ch, '').upper()
    return content


def init_combinations():
    genom_combinations = {}
    all_combinations = itertools.product('ACGT', repeat=4)
    for comb in all_combinations:
        comb = ''.join(comb)
        if comb in genom_combinations or reverse_complement(comb) in genom_combinations:
            continue
        genom_combinations[comb] = 0
    return genom_combinations


def compute_features(sequence, step=1, init=None, use_tqdm=False):
    # Computes counts for all features for given sequence 
    # (also includes N characters which are disposed of later)
    
    if init is None:
        init = init_combinations()
    c = Counter(init)
    
    if 'N' in sequence:
        return c, 0
            
    progress_bar = tqdm if use_tqdm else lambda f: f
    for start_i in progress_bar(range(0, len(sequence)-4, step)):
        gen = sequence[start_i:start_i+4]
        if gen in c:
            c.update([gen])
        else:
            c.update([reverse_complement(gen)])
    return c, 1

In [3]:
data = load_fasta('data/chr21.fa')
test_start = data.find('A')
length = 1500
test_seq = data[test_start:test_start+length]
test_seq

'ATCCACCCGCCTTGGCCTCCTAAAGTGCTGGGATTACAGGTGTTAGCCACCACGTCCAGCTGTTAATTTTTATTTAATAAGAATGACAGAGTGAGGGCCATCACTGTTAATGAAGCCAGTGTTGCTCACAGCCTCCCCTTGGTCACTTTTTGTGACTGAAGGGCATGTGTTCAGGCAAGATTGTTGGGTGGCTGTGTTTTGTCTTCTTCCAGCTCGGCCATGGAATAGCCTGTGGGGACCTACTCTGTGGTCCCCAGGGAGCTACTCTGTGGGGGCTGTTTCTGTTCAGCAGGGAAGGCTCTGCCCTTGCTGTTAGCTCCTGGAGGGCTGCGGACGGCACCTGCTGTGTTCACAGATGACAGTTACTTCCCTAGGTAGTCTGCATGTTGGGCCTCCCAGGACTGGTTCTCTAAGGGCAATGTGAGGACAGACAGAAAAACCAAATTCTGCCAAAGTTTTTAAATAGGTTTATTCTGAGCCAATAAGAGTGACCATGGCCTGGGAAATACAGTCTTAAGAGATCCCGAGGAAGTGCACCTGAGGCGGTCAGTTACAATTTGGTTTTATGTATTTATTTATTTTTATTTTATTTATTTATTTATTTGTTTTTGAGACGGAGGCTTGCTCTGTCGCCCAGGCTGGAGTGCAGTGGCGTGATCTCGGCTCACCGCAAGCTCCGCCTCCCGCGTTCCTGCCATTCTCCTGCCTCAGCCTCCCGAGTAGCTGGGACTACAGGCGCCCGCCACCATGCCTGGCTAATTTTTTTATATTTTTAGTAGAGACGGGGTTTCACCGTGTTAGCCAGGATGGTCTCAATCTCCTGACCTCGTGATCCGCCCGCCTCTGCCTCCCAAAGTGCTGGGATTACAGGCGTGAGCCACCACTCCCAGCACAATTGGTTTTGTACATTTCAGGGAGATGCGAACTGCAGGTGGAATCAGAAAACAGTACACGGGCCGGGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGAGCT

In [4]:
len(init_combinations())

136

In [5]:
compute_features(test_seq)[0].most_common(10)

[('AAAA', 38),
 ('ACAG', 28),
 ('CAGG', 28),
 ('AGGC', 25),
 ('AGCC', 23),
 ('CCAG', 23),
 ('CCCA', 23),
 ('CAGC', 22),
 ('CTCA', 22),
 ('AAAT', 21)]

# Model

In [6]:
def load_examples(path, label):
    # loads vista1500 and random1500
    
    with open(path) as f:
        lines = f.readlines()
        
    examples = []
    for line in lines:
        line = line.strip()
        if not line or line.startswith('>'):
            continue
            
        else:
            sequence = line.upper()
            examples.append((sequence, label))
    return examples


def examples_to_df(examples):
    # computes features for each example from vista1500 and random1500
    
    init = init_combinations()
    rows = []
    for sequence, label in examples:
        count, _ = compute_features(sequence)
        count['label'] = label
        row = pd.Series(count)
        rows.append(row)
    df = pd.concat(rows, axis=1).T
    return df


def load_df(path, label):
    examples = load_examples(path, label)
    return examples_to_df(examples)

In [7]:
df_true = load_df('data/vista1500', label=1)
df_fake = load_df('data/randoms1500', label=0)
df = pd.concat([df_true, df_fake], axis=0, ignore_index=True).sample(frac=1).reset_index(drop=True)
df.head()

,AAAA,AAAC,AAAG,AAAT,AACA,AACC,AACG,AACT,AAGA,AAGC,...,TACA,TAGA,TATA,TCAA,TCCA,TCGA,TGAA,TGCA,TTAA,label
0,36,21,21,28,18,12,1,24,16,10,...,12,12,6,13,18,0,21,9,12,1
1,72,25,24,41,27,5,7,22,19,11,...,13,8,8,18,15,2,17,11,12,1
2,68,29,22,53,24,11,1,18,13,6,...,17,15,15,25,18,0,18,7,21,0
3,30,21,21,21,13,14,5,15,8,18,...,15,5,0,16,18,1,22,8,5,1
4,31,15,25,23,15,7,0,12,20,17,...,5,10,4,11,20,0,16,7,10,1


In [8]:
X, y = df.drop('label', axis=1), df['label']

In [9]:
model = RandomForestClassifier()
results = cross_validate(model, X, y, scoring='roc_auc', cv=5)

In [10]:
results

{'fit_time': array([0.60043263, 0.609828  , 0.60021353, 0.60774541, 0.6054399 ]),
 'score_time': array([0.01368856, 0.01460171, 0.01449442, 0.01401281, 0.01362157]),
 'test_score': array([0.84986105, 0.8787749 , 0.86245947, 0.83595415, 0.85476586])}

In [11]:
model.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Dividing chromosome

In [12]:
def chromosome_features(chromosome, step, seq_length):
    # compute features for all sequences in the chromosome
    init = init_combinations()
    rows = []
    for seq_start in tqdm(range(0, len(chromosome) - seq_length, step)):
        sequence = chromosome[seq_start:seq_start+seq_length]
        count, correct = compute_features(sequence, step=1, init=init)
        count['correct'] = correct
        row = pd.Series(count)
        rows.append(row)
    df = pd.concat(rows, axis=1).T
    return df


data = load_fasta('data/chr21.fa')
chr_features = chromosome_features(data, step=750, seq_length=1500)

In [13]:
X, corrects = chr_features.drop('correct', axis=1), chr_features['correct']
preds = model.predict_proba(X)[:, 1]
preds.shape, preds[1]

((62278,), 0.46)

In [14]:
# adjust mean predictions for sequences containing N
mean_prediction = np.mean(preds[corrects == 1])
preds[corrects == 0] = mean_prediction

In [15]:
def write_wig(preds, save_path):
    with open(save_path, 'w') as f:
        f.write('fixedStep chrom=chr21 start=0 step=750 span=1500\n')
        for pred in preds:
            f.write(f'{pred}\n')
            
write_wig(preds, save_path='data/results.wig')